In [1]:
import sys

!{sys.executable} -m pip install langchain openai tqdm

In [2]:
OPENAI_API_KEY = "needed for using the model"

In [7]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [35]:
system_prompt = """take the task provided and change it into a psudo code. return only the code.
"""

my_magic_prompt = """{task}"""

In [36]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_magic_prompt),
])

In [37]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain_of_code = prompt | llm | output_parser

In [38]:
import pandas as pd

test = pd.read_csv('train_with_translation.csv')

In [108]:
sample = test.translation.sample()
sample = sample.values[0]
sample

'The digits of a four-digit number that is a multiple of 5 were written in reverse order, resulting in a second four-digit number. Then, the second number was subtracted from the original number, yielding 2457. Indicate any one such original number in your answer.'

In [116]:
code = chain_of_code.invoke({"task": sample})
code

'Let\'s denote the original four-digit number as "abcd" where a, b, c, and d represent the thousands, hundreds, tens, and units digits respectively.\n\n1. Reverse the digits of the original number to obtain a new number "dcba".\n2. Subtract the new number from the original number to get the result 2457.\n\nPseudocode:\n```\noriginalNumber = 1000*a + 100*b + 10*c + d\nnewNumber = 1000*d + 100*c + 10*b + a\n\nresult = originalNumber - newNumber\n\nif result = 2457:\n    return originalNumber\n```\n\nNote: This pseudocode does not include any looping or exhaustive searching. It simply represents the steps required to find one such original number that satisfies the given conditions.'

In [117]:
answer_prompt = """
code:
{code}
get the final answer using the code, step by step like the code. end with a line what the final answer is, don't approximate.
"""

In [118]:
prompt_ans = ChatPromptTemplate.from_messages([
    ("system", answer_prompt),
    ("user", my_magic_prompt),
])
chain_answer = prompt_ans | llm | output_parser

In [119]:
answer = chain_answer.invoke({
    "task": sample,
    "code": code,
})
answer

'Let\'s go step by step to find the original number.\n\n1. Reverse the digits of the original number to obtain a new number "dcba":\n   - Reverse the digits of 2457: 7542\n\n2. Subtract the new number from the original number to get the result 2457:\n   - Original number - New number = 2457\n   - Let\'s denote the original number as "abcd" and the new number as "dcba":\n   - 1000*a + 100*b + 10*c + d - (1000*d + 100*c + 10*b + a) = 2457\n   - Simplifying the equation, we get:\n   - 999*a - 999*d + 90*b - 90*c = 2457\n   - Divide both sides of the equation by 9 to simplify further:\n   - 111*a - 111*d + 10*b - 10*c = 273\n\nWe need to find values for a, b, c, and d that satisfy the equation 111*a - 111*d + 10*b - 10*c = 273.\n\nOne possible solution is:\na = 3\nb = 0\nc = 2\nd = 9\n\nLet\'s substitute these values back into the original number and verify if it satisfies the conditions:\nOriginal number = 1000*a + 100*b + 10*c + d = 1000*3 + 100*0 + 10*2 + 9 = 3209\n\nNow, let\'s check i

In [120]:
extract_answer_prompt = """
answer: {answer}
get only the final answer as float, no other charcter but float is allowed. if there multiple numbers concat them.
"""

In [121]:
prompt_ans_ext = ChatPromptTemplate.from_messages([
    ("system", extract_answer_prompt),
    ("user", """"""),
])
chain_answer_ext = prompt_ans_ext | llm | output_parser

In [122]:
ext_answer = chain_answer_ext.invoke({
    "answer": answer,
})
ext_answer

'The final answer is that there is no original four-digit number that satisfies the given conditions.'

In [106]:
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm

tqdm.pandas()
test[['answer', 'solution']] = None

def get_answer_llm(translation, chain_answer_ext=chain_answer_ext, chain_answer=chain_answer):
    code = chain_of_code.invoke({"task": sample})

    answer = chain_answer.invoke({
    "task": sample,
    "code": code,
    })
    ext_answer = chain_answer_ext.invoke({
    "answer": answer,
    })
    return pd.Series({'ext_answer': ext_answer, 'answer': answer})

with get_openai_callback() as cb:
    test.loc[test['answer'].isna(), ['ext_answer', 'answer']] = (
        test.loc[test['answer'].isna(), 'translation'].sample(3).progress_apply(get_answer_llm)
    )
    print(cb)

100%|█████████████████████████████████████████████| 3/3 [01:37<00:00, 32.48s/it]

Tokens Used: 5300
	Prompt Tokens: 3554
	Completion Tokens: 1746
Successful Requests: 9
Total Cost (USD): $0.008823


In [107]:

test[~test.answer.isna()]

,problem_id,problem_text,answer,hint,translation,solution,ext_answer
59,10974,"В параллелограмме $ABCD$ известно, что $AB=6$,...","For each function, we need to choose the corre...","прямоугольник, теорема пифагора, площадь","In parallelogram $ABCD$, it is known that $AB=...",None,"3, 4, 1, 2"
74,7113,"Набор полотенец, который стоил 280 рублей, про...","First, let's match the functions with their ch...",процент от числа,A set of towels that cost 280 rubles is being ...,None,1432
94,13550,Расстояние между городами A и B равно 470 км. ...,The code provided is trying to match the funct...,подсчеты,The distance between cities A and B is 470 km....,None,1.2.3.4
